In [ ]:
import random
from Utility import df_ops, music_ops, vmd_ops, io_ops, interpolate, cloud_ops
import pandas as pd
import os 
import pinecone
import pymeshio.common
import numpy as np

MIN_FRAMES = 30
MAX_FRAMES = 60
PAST_SONG_REPEAT_DISTANCE = 3
FILE_NAME = "shortNsweet"
BUFFER = 4

class DanceClip:
  def __init__(self, pathToVmdFolder, startFrame, numberOfFrames):
    self.pathToVmdFolder = pathToVmdFolder
    self.startFrame = startFrame
    self.numberOfFrames = numberOfFrames
  
  def __str__(self):
    return "DanceClip: " + self.pathToVmdFolder + " startFrame: " + str(self.startFrame) + " numberOfFrames: " + str(self.numberOfFrames)

def queryForSimilarPoses(pinecone, vector, songNamesToExclude):
  index = pinecone.Index(os.getenv('PINECONE_INDEX_NAME'))
  songs = index.query(
    vector=vector,
    filter={
      "animationName": { "$nin": songNamesToExclude[-PAST_SONG_REPEAT_DISTANCE:] }
    },
    top_k=1,
    include_metadata=True
  )
  matchedSongName = songs.matches[0].metadata['animationName']
  frameNumberOfMatchedSong = songs.matches[0].metadata['frameNumber']
  distance = songs.matches[0]['score']
  print("found match: " + matchedSongName + " at frame: " + str(frameNumberOfMatchedSong) + " with distance: " + str(distance))
  return matchedSongName, frameNumberOfMatchedSong

def appendFramesPreservingCenter(df, dfToAppend, buffer=0):
  if df.empty:
    lastPositionOfDf = pymeshio.common.Vector3(0, 0, 0)
  else:
    lastPositionOfDfIdx = df[df['name'] == 'センター']['frame'].idxmax()
    lastPositionOfDf = df.iloc[lastPositionOfDfIdx]['position']

  try:
    firstPositionOfDfToAppendIdx = dfToAppend[dfToAppend['name'] == 'センター']['frame'].idxmin()
    firstPositionOfDfToAppend = dfToAppend.iloc[firstPositionOfDfToAppendIdx]['position']
  except:
    if dfToAppend.empty:
      print("dfToAppend is empty")
    if len(dfToAppend[dfToAppend['name'] == 'センター']) == 0:
      print("No rows with name 'センター' in dfToAppend")
    print("Index to access:", firstPositionOfDfToAppendIdx)
    print("Length of dfToAppend:", len(dfToAppend))
    raise Exception("Error")

  differenceInPos = firstPositionOfDfToAppend - lastPositionOfDf
  differenceInPos.y = 0

  positionBoneNames = ['センター', '右足ＩＫ', '左足ＩＫ']
  dfToAppendWithAdjustedPositions = dfToAppend.copy()
  dfToAppendWithAdjustedPositions['position'] = np.where(dfToAppendWithAdjustedPositions['name'].isin(positionBoneNames), 
                                  dfToAppendWithAdjustedPositions['position'] - differenceInPos,
                                  dfToAppendWithAdjustedPositions['position'])
  return df_ops.appendFrames(df, dfToAppendWithAdjustedPositions, buffer=buffer)

def getVectorFromVectorId(pinecone, songName, framenumber):
  vectorId = songName + "-" + str(int(framenumber))
  index = pinecone.GRPCIndex(os.getenv('PINECONE_INDEX_NAME'))
  fetchVectorId = index.fetch([vectorId])
  return fetchVectorId['vectors'][vectorId]['values']

def generateAnimationClips(numberOfFrames):
  INPUT_FOLDER = 'inputData'
  pinecone = cloud_ops.initPinecone()
  sum = 0
  end = numberOfFrames
  newSongName = random.choice([entry for entry in os.listdir(INPUT_FOLDER) if os.path.isdir(os.path.join(INPUT_FOLDER, entry))])
  print("starting with song: " + newSongName)
  pastSongs = [newSongName]
  animationClips = []
  frameNumberOfMatchedSong = None

  while sum < end:
    if sum != 0:
      print("querying for similar poses for song: " + prevSongName + " at frame: " + str(startingFrameOfNewFrames + numberOfNewFramesToAdd))
      poseVector = getVectorFromVectorId(pinecone, prevSongName, startingFrameOfNewFrames + numberOfNewFramesToAdd)
      newSongName, frameNumberOfMatchedSong = queryForSimilarPoses(pinecone, poseVector, pastSongs)
      pastSongs.append(newSongName)

    vmd = [f for f in os.listdir(INPUT_FOLDER + "\\" + newSongName) if f.endswith('.vmd')][-1]
    df = vmd_ops.getDfFromVmdFileName(INPUT_FOLDER + "\\" + newSongName + "\\" + vmd)
    numberOfNewFramesToAdd = random.randint(MIN_FRAMES, MAX_FRAMES)
    if frameNumberOfMatchedSong and frameNumberOfMatchedSong + numberOfNewFramesToAdd > df_ops.getLastFrame(df): #handle if not enough frames left in song
      numberOfNewFramesToAdd = df_ops.getLastFrame(df) - frameNumberOfMatchedSong
      print("[ERROR] ---------- not enough frames left in song, reducing numberOfNewFramesToAdd to: " + str(numberOfNewFramesToAdd))
    startingFrameOfNewFrames = frameNumberOfMatchedSong if sum != 0 else random.randint(0, df_ops.getLastFrame(df) - numberOfNewFramesToAdd)

    animationClips.append(DanceClip(INPUT_FOLDER + "\\" + newSongName, startingFrameOfNewFrames, numberOfNewFramesToAdd))
    sum += numberOfNewFramesToAdd
    prevSongName = newSongName
  
  return animationClips

def generateVmdAnimationFromDance(animationClips, savePath, saveName, bufferFrames=0, viewInterpolation=False):
  newAnimation = pd.DataFrame()
  newInterpolatedAnimation = pd.DataFrame()
  frameNumber = 0

  for animationClip in animationClips:
    vmd = [f for f in os.listdir(animationClip.pathToVmdFolder) if f.endswith('.vmd')][-1]
    df = vmd_ops.getDfFromVmdFileName(animationClip.pathToVmdFolder + "\\" + vmd)
    newAnimation = appendFramesPreservingCenter(newAnimation, df_ops.parseFramesFromDf(df, animationClip.startFrame, animationClip.startFrame + animationClip.numberOfFrames), bufferFrames)
    if viewInterpolation:
      interpolatedDf = df_ops.loadDfFromFeather(animationClip.pathToVmdFolder + "\\interpolatedMotion.feather")
      newInterpolatedAnimation = df_ops.appendFrames(newInterpolatedAnimation, df_ops.parseFramesFromDf(interpolatedDf, animationClip.startFrame, animationClip.startFrame + animationClip.numberOfFrames), bufferFrames)
    print("adding song: " + animationClip.pathToVmdFolder.split("\\")[-1] + "-" + str(animationClip.startFrame) + " at frame: " + str(frameNumber))
    frameNumber += animationClip.numberOfFrames + bufferFrames

  vmd_ops.saveDfToVmdFile(newAnimation, savePath + "\\" + saveName + ".vmd")
  if viewInterpolation:
    vmd_ops.saveDfToVmdFile(newInterpolatedAnimation, savePath + "\\" + saveName + "-interpolated-DEBUG.vmd")

  return newAnimation

In [5]:
def generateAnimationClips_fromTimestamps(timestamps):
  framesArray = music_ops.get_framesArray_from_movesArray(timestamps)
  INPUT_FOLDER = 'inputData'
  pinecone = cloud_ops.initPinecone()
  sum = 0
  newSongName = random.choice([entry for entry in os.listdir(INPUT_FOLDER) if os.path.isdir(os.path.join(INPUT_FOLDER, entry))])
  print("starting with song: " + newSongName)
  pastSongs = [newSongName]
  animationClips = []
  frameNumberOfMatchedSong = None

  prevFrameNumberOfMove = framesArray[0]
  for i in range(len(framesArray)):
    print("on move: " + str(i) + " of " + str(len(framesArray)))
    if sum != 0:
      print("querying for similar poses for song: " + prevSongName + " at frame: " + str(startingFrameOfNewFrames + numberOfNewFramesToAdd))
      poseVector = getVectorFromVectorId(pinecone, prevSongName, startingFrameOfNewFrames + numberOfNewFramesToAdd)
      newSongName, frameNumberOfMatchedSong = queryForSimilarPoses(pinecone, poseVector, pastSongs)
      pastSongs.append(newSongName)

    vmd = [f for f in os.listdir(INPUT_FOLDER + "\\" + newSongName) if f.endswith('.vmd')][-1]
    df = vmd_ops.getDfFromVmdFileName(INPUT_FOLDER + "\\" + newSongName + "\\" + vmd)
    numberOfNewFramesToAdd = framesArray[i] - prevFrameNumberOfMove
    if frameNumberOfMatchedSong and frameNumberOfMatchedSong + numberOfNewFramesToAdd > df_ops.getLastFrame(df): #handle if not enough frames left in song
      newNumberOfNewFramesToAdd = df_ops.getLastFrame(df) - frameNumberOfMatchedSong
      print("[ERROR] ---------- not enough frames left in song, reducing numberOfNewFramesToAdd from: " + str(numberOfNewFramesToAdd) +" to: " + str(newNumberOfNewFramesToAdd))
      numberOfNewFramesToAdd = newNumberOfNewFramesToAdd
    startingFrameOfNewFrames = frameNumberOfMatchedSong if sum != 0 else random.randint(0, df_ops.getLastFrame(df) - numberOfNewFramesToAdd)

    animationClips.append(DanceClip(INPUT_FOLDER + "\\" + newSongName, startingFrameOfNewFrames, numberOfNewFramesToAdd))
    sum += numberOfNewFramesToAdd
    prevSongName = newSongName
    prevFrameNumberOfMove = framesArray[i]
  
  return animationClips

In [3]:
from spleeter.separator import Separator
import librosa

def filterMinInBetweenFrames(arr, numInBetween):
    if not arr:
        return []

    # Initialize the last added element as the first element of the array
    last_added = arr[0]
    filtered = [last_added]

    for num in arr[1:]:
        if num >= last_added + numInBetween:
            filtered.append(num)
            last_added = num

    return filtered

def generateMovesTimestampsFromAudio(inputAudioFolder, inputAudioFilename, outputAudioFolder):
  # Initialize separator in '2stems' mode.
  separator = Separator('spleeter:2stems')

  # Perform the separation.
  separator.separate_to_file(inputAudioFolder + "/" + inputAudioFilename + ".mp3", outputAudioFolder)
  VocalTrackFilePath = outputAudioFolder + "/" + inputAudioFilename + "/vocals.wav"

  significantAudioHits = music_ops.get_significant_change_times_from_audio(VocalTrackFilePath)

  movesArray = filterMinInBetweenFrames(significantAudioHits, 1)
  return movesArray

  
INPUT_AUDIO_FOLDER = 'inputAudio'
INPUT_AUDIO_FILENAME = 'marine'
OUTPUT_AUDIO_FOLDER = 'outputAudio'
movesFrameNumberArray = generateMovesTimestampsFromAudio(INPUT_AUDIO_FOLDER, INPUT_AUDIO_FILENAME, OUTPUT_AUDIO_FOLDER)

INFO:tensorflow:Using config: {'_model_dir': 'pretrained_models\\2stems', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.7
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use output_signature instead
Instructions for updating:
Use output_signature instead
INFO:tensorflow:Calling model_fn.
INFO:tensorfl

In [6]:
animationClips = generateAnimationClips_fromTimestamps(movesFrameNumberArray)
# animationDf = generateVmdAnimationFromDance(animationClips, 'outputMotions', FILE_NAME, bufferFrames=BUFFER, viewInterpolation=True)

starting with song: PV626 - Weekender Girl
on move: 0 of 103
on move: 1 of 103
on move: 2 of 103
querying for similar poses for song: PV626 - Weekender Girl at frame: 2967
found match: PV272 - Hibana at frame: 1873.0 with distance: 0.992476344
on move: 3 of 103
querying for similar poses for song: PV272 - Hibana at frame: 1924.0
found match: PV224 - Rosary Pale at frame: 4638.0 with distance: 0.98892957
on move: 4 of 103
querying for similar poses for song: PV224 - Rosary Pale at frame: 4770.0
found match: PV239 - Tricolore Airline at frame: 3891.0 with distance: 0.974899471
on move: 5 of 103
querying for similar poses for song: PV239 - Tricolore Airline at frame: 4033.0
found match: PV626 - Weekender Girl at frame: 3622.0 with distance: 0.989567101
on move: 6 of 103
querying for similar poses for song: PV626 - Weekender Girl at frame: 3903.0
found match: PV049 - PoPiPo at frame: 1246.0 with distance: 0.974194407
on move: 7 of 103
querying for similar poses for song: PV049 - PoPiPo at 

In [8]:
animationDf = generateVmdAnimationFromDance(animationClips, 'outputMotions', FILE_NAME, bufferFrames=BUFFER, viewInterpolation=True)

adding song: PV626 - Weekender Girl-3905 at frame: 0
adding song: PV626 - Weekender Girl-2893 at frame: 4
adding song: PV272 - Hibana-1873.0 at frame: 82
adding song: PV224 - Rosary Pale-4638.0 at frame: 137
adding song: PV239 - Tricolore Airline-3891.0 at frame: 273
adding song: PV626 - Weekender Girl-3622.0 at frame: 419
adding song: PV049 - PoPiPo-1246.0 at frame: 704
adding song: PV906 - Slow Motion-4815.0 at frame: 751
adding song: PV095 - Nekomimi Switch-1116.0 at frame: 792
adding song: PV922 - Amazing Dolce-4761.0 at frame: 838
adding song: PV261 - Kimi no Taion-4398.0 at frame: 987
adding song: PV250 - Nice to Meet You, Mr. Earthling-5484.0 at frame: 1033
adding song: PV255 - Skeleton Orchestra and Lilia-2206.0 at frame: 1082
adding song: PV902 - Aidee-2893.0 at frame: 1264
adding song: PV312 - Two-Faced Lover Live Mode-5571.0 at frame: 1301
adding song: PV029 - Requiem For The Phantasma-3920.0 at frame: 1379
adding song: PV222 - Black Gold-689.0 at frame: 1542
adding song: PV

In [ ]:
music_ops.get_beeptrack_from_movesArray(movesFrameNumberArray, "yess.wav")

In [ ]:
pinecone = cloud_ops.initPinecone()
index = pinecone.Index(os.getenv('PINECONE_INDEX_NAME'))
print(index.fetch([str('PV305 - Just be Friends Live Mode-5510')]))

In [ ]:
import os
import wave
from scipy.signal import resample

def create_empty_wav(output_file):
  with wave.open(output_file, 'wb') as out_wav:
    out_wav.setparams((1, 2, 44100, 0, 'NONE', 'not compressed'))
    out_wav.writeframes(b'')

def create_empty_wav_with_params(input_file, output_file):
  with wave.open(input_file, 'rb') as wav:
    params = wav.getparams()

  with wave.open(output_file, 'wb') as out_wav:
    out_wav.setparams(params)
    out_wav.writeframes(b'')

def append_wav_files(input_file1, input_file2, output_file, start_second, end_second):
  with wave.open(input_file1, 'rb') as wav1, wave.open(input_file2, 'rb') as wav2:
    params1 = wav1.getparams()
    params2 = wav2.getparams()

    frames1 = wav1.readframes(wav1.getnframes())
    frames1 = np.frombuffer(frames1, dtype=np.int16)

    # Calculate the start and end frames for the segment in file2
    frame_rate1 = wav1.getframerate()
    frame_rate2 = wav2.getframerate()
    start_frame = int(start_second * frame_rate2)
    end_frame = int(end_second * frame_rate2)
    n_frames = end_frame - start_frame

    # Position the file pointer and read the segment frames
    wav2.setpos(start_frame)
    frames2 = wav2.readframes(n_frames)
    frames2 = np.frombuffer(frames2, dtype=np.int16)

    # Resample frames2 to match the frame rate of frames1
    resampled_frames2 = resample(frames2, int(len(frames2) * frame_rate1 / frame_rate2)).astype(np.int16)

    # Combine frames1 and resampled_frames2
    combined_frames = np.concatenate((frames1, resampled_frames2))

    with wave.open(output_file, 'wb') as out_wav:
      out_wav.setparams(params1)
      out_wav.writeframes(combined_frames.tobytes())

import wave
import array

def append_silence(input_file, output_file, silence_duration):
  with wave.open(input_file, 'rb') as wav:
    params = wav.getparams()
    frames = wav.readframes(wav.getnframes())

    # Create 1 second of silence
    frame_rate = params.framerate
    n_channels = params.nchannels
    sampwidth = params.sampwidth

    n_frames_silence = int(frame_rate * silence_duration)
    silence_frames = array.array("h", [0] * n_frames_silence * n_channels)

    # Convert the array to bytes
    silence_frames = silence_frames.tobytes()

    with wave.open(output_file, 'wb') as out_wav:
      out_wav.setparams(params)
      out_wav.writeframes(frames + silence_frames)

def get_wav_duration(wav_file_path):
  with wave.open(wav_file_path, 'rb') as wav_file:
    n_frames = wav_file.getnframes()
    frame_rate = wav_file.getframerate()
    duration = n_frames / float(frame_rate)
  return duration

def pullAudioFromAnimationClips(animationClips, savePath, saveName, buffer=0):
  totalFrames = 6
  newWaveName = savePath + "\\" + saveName + ".wav"
  createOutputFile = True
  for animationClip in animationClips:
    songWavName = [f for f in os.listdir(animationClip.pathToVmdFolder) if f.endswith('.wav')][-1]
    songWav = animationClip.pathToVmdFolder + "\\" + songWavName
    print(songWav)
    if createOutputFile:
      create_empty_wav_with_params(songWav, newWaveName)
      append_silence(newWaveName, newWaveName, 6 / 30) # initial silence
      createOutputFile = False
    startSecond = animationClip.startFrame / 30
    endSecond = (animationClip.startFrame + animationClip.numberOfFrames) / 30
    append_wav_files(newWaveName, songWav, newWaveName, startSecond, endSecond)
    totalFrames += animationClip.numberOfFrames
    print("added: " + str(endSecond - startSecond) + " seconds for " + str(animationClip.numberOfFrames) + " frames. Total frames now at: " + str(totalFrames) + " frames. Total calculated seconds: " + str(totalFrames / 30) + ". Total actual seconds now at: " + str(get_wav_duration(newWaveName)) + " seconds.")
    append_silence(newWaveName, newWaveName, buffer / 30)
    totalFrames += buffer

pullAudioFromAnimationClips(animationClips, 'outputMotions', FILE_NAME, buffer=BUFFER)

In [ ]:
# multiply x and y quaternion by -1 and replace corresponding bone to flip aniamtion